## 🌾 Question 02: NumPy Data Cleaning and Normalization

**Goal:** Clean and normalize a large, messy CSV file of soil moisture sensor readings using the high-performance capabilities of NumPy.
**Topics:** File I/O, NumPy Arrays, Vectorized Operations, Boolean Indexing, Axis Operations, Broadcasting, Min-Max Normalization.

### Data Overview
* The `sensor_data.csv` file has **100 columns** (one per sensor) and **720 rows** (one per hour).
* The data contains missing values (`"-999"`), impossible values (negative or greater than 100), and is stored as strings.

### Your Task:

1.  **Load Data:** Use **standard Python file I/O** (no Pandas) to read `sensor_data.csv` line-by-line, split the commas, and load it into a Python list of lists.
2.  **NumPy Conversion:** Convert this nested list into a **2D NumPy array**.
3.  **Data Cleaning (Vectorized):** Use NumPy's vectorized operations (no for loops) to:
    * Replace all occurrences of `"-999"` with **`np.nan`**.
    * Use **Boolean indexing** to find and replace all values **less than 0** and all values **greater than 100** with **`np.nan`**.
4.  **Data Analysis (Axis Operations):**
    * Calculate the **average (mean) moisture** for each sensor (**column-wise**), **ignoring `np.nan` values**.
    * Calculate the **median moisture** for each hour (**row-wise**), **ignoring `np.nan` values**.
    * Identify the sensor (**column index**) with the **highest number of invalid readings** (the `np.nan` values you just set).
5.  **Normalization (Broadcasting):**
    * Find the minimum ($X_{min}$) and maximum ($X_{max}$) valid reading across the **entire dataset**.
    * Use **NumPy broadcasting** to perform **Min-Max Normalization** on the entire array, scaling all data to be between 0 and 1. The formula is:
        $$X_{norm}=\frac{X-X_{min}}{X_{max}-X_{min}}$$
6.  **Save Output:** Write the final, cleaned, and normalized NumPy array to a new CSV file named **`sensor_data_normalized.csv`**.